<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [0]:
def collect_test_labels(_data_gen_test, _data_out, classification_mode, quick_test):
    # Collecting ground truth for test data
    nb_batch = 2 if quick_test else _data_gen_test.get_total_batches_in_data()

    batch_size = _data_out[0][0]
    gt_sed = np.zeros((nb_batch * batch_size, _data_out[0][1], _data_out[0][2]))
    gt_doa = np.zeros((nb_batch * batch_size, _data_out[0][1], _data_out[1][2]))

    print("nb_batch in test: {}".format(nb_batch))
    cnt = 0
    for tmp_feat, tmp_label in _data_gen_test.generate():
        gt_sed[cnt * batch_size:(cnt + 1) * batch_size, :, :] = tmp_label[0]
        gt_doa[cnt * batch_size:(cnt + 1) * batch_size, :, :] = tmp_label[1]
        cnt = cnt + 1
        if cnt == nb_batch:
            break
    return gt_sed.astype(int), gt_doa
    
def print_generic_evaluation(model, _dataset, _ov, _split):
    data_gen_test = DataGenerator(
            dataset=_dataset, ov=_ov, split=_split, db=params['db'], nfft=params['nfft'],
            batch_size=params['batch_size'], seq_len=params['sequence_length'], classifier_mode=params['mode'],
            weakness=params['weakness'], datagen_mode='test', cnn3d=params['cnn_3d'], xyz_def_zero=params['xyz_def_zero'],
            azi_only=params['azi_only'], shuffle=False
        )
    
    values = model.evaluate_generator(data_gen_test.generate(),
                                steps=2 if params['quick_test'] else data_gen_test.get_total_batches_in_data(),
                                verbose = 1)
    print("Accuracy and Loss evaluation for dataset", _dataset,
          "ov", _ov, "split", _split)
    i = 0
    for name in model.metrics_names:
        print(name, values[i])
        i+=1

def print_sed_doa_scores(model, _dataset, _ov, _split):
    data_gen_test = DataGenerator(
            dataset=_dataset, ov=_ov, split=_split, db=params['db'], nfft=params['nfft'],
            batch_size=params['batch_size'], seq_len=params['sequence_length'], classifier_mode=params['mode'],
            weakness=params['weakness'], datagen_mode='test', cnn3d=params['cnn_3d'], xyz_def_zero=params['xyz_def_zero'],
            azi_only=params['azi_only'], shuffle=False
        )

    gt = collect_test_labels(data_gen_test, data_out, params['mode'], params['quick_test'])
    sed_gt = reshape_3Dto2D(gt[0])
    doa_gt = reshape_3Dto2D(gt[1])

    pred = model.predict_generator(
                generator=data_gen_test_1.generate(),
                steps=2 if params['quick_test'] else data_gen_test.get_total_batches_in_data(),
                verbose=1
            )

    sed_pred = reshape_3Dto2D(pred[0]) > 0.5
    doa_pred = reshape_3Dto2D(pred[1])

    sed_scores = compute_sed_scores(sed_pred, sed_gt, data_gen_test.nb_frames_1s())
    if params['azi_only']:
        doa_scores, conf_mat = compute_doa_scores_regr_xy(doa_pred, doa_gt, sed_pred, sed_gt)
    else:
        doa_scores, conf_mat = compute_doa_scores_regr_xyz(doa_pred, doa_gt, sed_pred, sed_gt)
        
    #print('Confusion Mx : {}'.format(conf_mat))
    #print('best_conf_mat_diag : {}'.format(np.diag(best_conf_mat)))
    print('SED Metrics: ER_overall: {}, F1_overall: {}'.format(sed_scores[0], sed_scores[1]))
    print('DOA Metrics: doa_loss_gt: {}, doa_loss_pred: {}, good_pks_ratio: {}'.format(
        doa_scores[1], doa_scores[2], doa_scores[5] / float(sed_gt.shape[0])))
    
print_generic_evaluation(model, 'ansim', 1, 1)
#print_generic_evaluation(model, 'ansim', 1, 2)
#print_generic_evaluation(model, 'ansim', 1, 3)
print_sed_doa_scores(model, 'ansim', 1, 1)
#print_sed_doa_scores(model, 'ansim', 1, 2)
#print_sed_doa_scores(model, 'ansim', 1, 3)
